 The output has been shown for first 3 game_id's due to computational constraints. The output for all the games can be obtained by running the same code over the entire dataset.

1. The calculation startegy needs to cater to the loss in data.

2. We first find out the unique number of games, by calculating the unique game_id's in the data, then we go on to find for each    such game the sessions (total and valid) and the average session time.

3. Once we have isolated the game_id's, we isolate users characterized by ai5. Now each unique ai5 is a user. Due to uncaptured    entries we have start or stop events without corresponding stop or star events respectively. Therefore we consider two          consecutive entries as a session assuming between these two we have not had any uncaptured sessions and neglecting data that    has not been captured completely.

4. Insights about the users for game id =:
      Total no. of users : 
      User with maximum number of sessions:
      User with minimum number of sessions:
      User with maximum total length of session:
      User with minimum total length of session:
   
5. Discrepancies in data:
    i)  We do not have corresponding stop/start event for each start/stop event:
        Way around: Finding a suitable model for start vs stop times and using the same to find out the missing event time.



In [129]:
# Importing libraries
import pandas as pd
import json
import datetime
import matplotlib.pyplot as plt
import numpy as np
import time
from sklearn import datasets, linear_model
import csv


In [ ]:
# Loading datasets

with open('ggevent.log') as data:
    data = data.readlines()
data_new = [json.loads(x)  for x in data ]

In [ ]:
# Formatting dataset

with open('ggevent_new.csv', 'w') as csvfile:
    dict_temp = dict(data_new[0]['bottle'],  **data_new[0]['post'], **data_new[0]['headers'])
    writer = csv.DictWriter(csvfile, dict_temp.keys())
    writer.writeheader()
    [writer.writerow(dict(data_new[i]['bottle'],  **data_new[i]['post'], **data_new[i]['headers'])) for i in range(0, len(data))]
dn = pd.read_csv('ggevent_new.csv')


In [ ]:
# Defining Variables
total_session = [] #stores total number of sessions for each game
valid_session = [] #stores total number of valid sessions for each game
average_session = [] #stores average length of valid sessions for each game

#temporary variables
temp_length = []
temp_number = []

#user insight variables
user_max_length = [] #for each game the user with max session length
user_min_length = [] #for each game the user with min session length
user_max_number = [] #for each game the user with max number of session
user_min_number = [] #for each game the user with min number of session

#temporary variables
user_max_length_session = []
user_min_length_session = []
user_max_number_session = []
user_min_number_session = []

In [130]:
len_game_id_unique = len(dn.game_id.unique()) #finding total number of games
for i in range(0,len_game_id_unique) currently running for first 3 game_id's
    #definging temporary variables
    
    temp_total_session = 0
    temp_valid_session = 0
    temp_average_session = 0
    
    # Slicing the dataset to obtain users for each game_id
    game_id_unique_slice = dn.loc[dn['game_id'] == game_id_unique[i]]
    #Finding all users for a given game_id
    game_id_ai5_unique = game_id_unique_slice.ai5.unique()
    len_ai5_unique = len(dn.ai5.unique())
    
    #iterating over users to find required stats
    for j in range(0,len_ai5_unique):
        
        #Slicing to obtain all sessions for a given user and a given game
        game_id_ai5_unique_slice = game_id_unique_slice .loc[game_id_unique_slice['ai5'] == dn.ai5.unique()[j]]
        
        #Finding complete sessions for the user
        game_id_ai5_unique_slice_new_start = pd.DataFrame([ row for row in game_id_ai5_unique_slice.itertuples()                                                   
                                                           if row[3] == 'ggstart' and 
                                                           game_id_ai5_unique_slice['event'].shift(-1)[row[0]] == 'ggstop' ])
        game_id_ai5_unique_slice_new_stop = pd.DataFrame([row for row in game_id_ai5_unique_slice.shift(-1).itertuples()                                             
                                                          if row[3] == 'ggstop' and game_id_ai5_unique_slice['event'][row[0]] == 'ggstart'  ])
        
        #Obtaining start and stop times to get stats
        if not game_id_ai5_unique_slice_new_start.empty:
            time_start = [datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S.%f") 
                         for i in game_id_ai5_unique_slice_new_start['timestamp']]
            time_stop = [datetime.datetime.strptime(i, "%Y-%m-%d %H:%M:%S.%f") 
                         for i in game_id_ai5_unique_slice_new_stop['timestamp']]
            
            #Regressing to be able to predict for incomplete session events
            x = np.asarray([time.mktime(i.timetuple()) for i in time_start]).reshape(len(time_start), 1)
            y = np.asarray([time.mktime(i.timetuple()) for i in time_stop]).reshape(len(time_start), 1)
            regr = linear_model.LinearRegression()
            regr.fit(x, y)
            plt.plot(x, regr.predict(x), color='blue', linewidth=3)
            
            #Finding length of sessions
            time_diff = [time_start[i] - time_stop[i-1] for i in range(1, len(time_start))]
            time_diff = [(i.seconds+i.microseconds/1000000) for i in time_diff]
            
            #Finding valid and totla sessions
            
            temp_valid_session = temp_valid_session +  len([i for i in time_diff  if i >= 60])
            temp_total_session = temp_total_session + len(time_diff) - len([i for i in time_diff if i <= 1])
            
            #Finding average length of sessions
            if temp_valid_session > 0 :
                temp_average_session =  temp_average_session + sum([i for i in time_diff if i>= 60])    
            else:
                temp_average_session = 0
                
            #Calculating user insights   
            if sum([i for i in time_diff  if i >= 60]) > sum(temp_length) :
                user_max_length_session = dn['ai5'][j]
            if sum([i for i in time_diff if i>= 60]) < sum(temp_length) :
                user_min_length_session = dn['ai5'][j]
            if len([i for i in time_diff if i >= 60]) > len(temp_number) :
                user_max_number_session = dn['ai5'][j]      
            if len([i for i in time_diff if i >= 60]) < len(temp_number) :
                user_min_number_session = dn['ai5'][j]
            temp_length = [i for i in time_diff if i >= 60]
            temp_number = [i for i in time_diff if i >= 60]
    
    #aggregating answers for all users and games
    valid_session = valid_session + [temp_valid_session]
    total_session = total_session + [temp_total_session]
    user_max_length = user_max_length + [user_max_length_session]
    user_min_length = user_min_length + [user_min_length_session]
    user_min_number = user_min_number + [user_min_number_session]
    user_max_number = user_max_number + [user_max_number_session]
    if temp_valid_session > 0:
        average_session = average_session + [temp_average_session/valid_session[i]]
    else:
        average_session = average_session + [0]

In [164]:
#outputing to a csv output.csv

with open('output.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerows([['average_session'] + average_session])
    wr.writerows([['total_session'] + total_session])
    wr.writerows([['valid_session']+ valid_session])
    wr.writerows([['user_max_length']+user_max_length])
    wr.writerows([['user_min_length']+user_min_length])
    wr.writerows([['user_max_number']+user_max_number])
    wr.writerows([['user_min_number']+user_min_number])
name = ['game_id'] + [str(i) for i in dn.game_id.unique()]
out = pd.read_csv('output.csv', names = name, header=None)
out

,game_id,55107008,10655437,30900473,18121481,32950170,43346372,19049241,10483946,40904844,...,63960431,21554188,60374084,98010070,85159873,82196685,94868282,35643983,26962869,59561974
0,average_session,519.1843744207952,729.7412995337129,373.90102877007695,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,total_session,32725,5022,1015,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,valid_session,3693,909,219,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,user_max_length,ed471817a37f964ae9904cd63d215418,ff6c39208bea7c6409fa164a2b800088,d23e73d5004ca552c345f6ac967febc6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,user_min_length,279cfceaa0ae700d830b24a216817e0e,d10d6beefa3a8a2bbb521d3e65b6ec0f,d9882d54e1fc954163ce64a900cd1a41,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,user_max_number,ed471817a37f964ae9904cd63d215418,016cf54e277fbb5fcbee781198464a8d,d23e73d5004ca552c345f6ac967febc6,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,user_min_number,279cfceaa0ae700d830b24a216817e0e,d10d6beefa3a8a2bbb521d3e65b6ec0f,d9882d54e1fc954163ce64a900cd1a41,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
